# Выбор локации для скважины

Мы работаем в добывающей компании «ГлавРосГосНефть». Необходимо решить, где бурить новую скважину.

Нам предоставлены пробы нефти в трёх регионах: в каждом 10 000 месторождений, где измерили качество нефти и объём её запасов. Необходимо построить модель машинного обучения, которая поможет определить регион, где добыча принесёт наибольшую прибыль. 
Проанализируем возможную прибыль и риски техникой *Bootstrap.*

Шаги для выбора локации:

- В избранном регионе ищут месторождения, для каждого определяют значения признаков;
- Строят модель и оценивают объём запасов;
- Выбирают месторождения с самым высокими оценками значений. Количество месторождений зависит от бюджета компании и стоимости разработки одной скважины;
- Прибыль равна суммарной прибыли отобранных месторождений.

## Загрузка и подготовка данных

In [1]:
# импортируем все необходимые библиотеки
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error as mse
import numpy as np

In [2]:
# зададим аргумент random_state через переменную rs
rs = 503350

In [3]:
# создадим датафрейм для каждого региона
geo_data_0 = pd.read_csv('C:\\Users\\503so\\OneDrive\\Desktop\\praktikum-to-git\\08_geo_data_0.csv')
geo_data_1 = pd.read_csv('C:\\Users\\503so\\OneDrive\\Desktop\\praktikum-to-git\\08_geo_data_1.csv')
geo_data_2 = pd.read_csv('C:\\Users\\503so\\OneDrive\\Desktop\\praktikum-to-git\\08_geo_data_2.csv')

In [4]:
# выведем на экран первые 5 строк таблиц для каждого из трех регионов
print('geo_data_0')
display(geo_data_0.head())
print()
print('geo_data_1')
display(geo_data_1.head())
print()
print('geo_data_2')
display(geo_data_2.head())

geo_data_0


,id,f0,f1,f2,product
0,txEyH,0.705745,-0.497823,1.221170,105.280062
1,2acmU,1.334711,-0.340164,4.365080,73.037750
2,409Wp,1.022732,0.151990,1.419926,85.265647
3,iJLyR,-0.032172,0.139033,2.978566,168.620776
4,Xdl7t,1.988431,0.155413,4.751769,154.036647



geo_data_1


,id,f0,f1,f2,product
0,kBEdx,-15.001348,-8.276000,-0.005876,3.179103
1,62mP7,14.272088,-3.475083,0.999183,26.953261
2,vyE1P,6.263187,-5.948386,5.001160,134.766305
3,KcrkZ,-13.081196,-11.506057,4.999415,137.945408
4,AHL4O,12.702195,-8.147433,5.004363,134.766305



geo_data_2


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746


Датасеты для трех разных регионов идентичны по своей структуре: 5 столбцов (признаков) и 100000 строк (объектов):

- id — уникальный идентификатор скважины;
- f0, f1, f2 — три признака точек (неважно, что они означают, но сами признаки значимы);
- product — объём запасов в скважине (тыс. баррелей).

In [5]:
# выведем информацию по каждому датасету
print('geo_data_0')
print(geo_data_0.info())
print()
print('geo_data_1')
print(geo_data_1.info())
print()
print('geo_data_2')
print(geo_data_2.info())

geo_data_0
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

geo_data_1
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dtypes: float64(4), object(1)
memory usage: 3.8+ MB
None

geo_data_2
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
id         100000 non-null object
f0         100000 non-null float64
f1         100000 non-null float64
f2         100000 non-null float64
product    100000 non-null float64
dt

In [6]:
# выведем на экран описание данных для каждого региона
print('geo_data_0')
print(geo_data_0.describe())
print()
print('geo_data_1')
print(geo_data_1.describe())
print()
print('geo_data_2')
print(geo_data_2.describe())

geo_data_0
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        0.500419       0.250143       2.502647      92.500000
std         0.871832       0.504433       3.248248      44.288691
min        -1.408605      -0.848218     -12.088328       0.000000
25%        -0.072580      -0.200881       0.287748      56.497507
50%         0.502360       0.250252       2.515969      91.849972
75%         1.073581       0.700646       4.715088     128.564089
max         2.362331       1.343769      16.003790     185.364347

geo_data_1
                  f0             f1             f2        product
count  100000.000000  100000.000000  100000.000000  100000.000000
mean        1.141296      -4.796579       2.494541      68.825000
std         8.965932       5.119872       1.703572      45.944423
min       -31.609576     -26.358598      -0.018144       0.000000
25%        -6.298551      -8.267985       1.000021   

Данные выглядят однородно и не содержат пропусков.

Выделим из датафреймов для каждого региона признаки `features` и целевой признак `target`. Признаки: *f0*, *f1* и *f2*, целевой признак - '*product*'. После чего разобъем переменные *features* и *target* на тренировочную и валидационную выборки. Для этого напишем функцию **df_split**, которая обращается к методу *train_test_split* библиотеки *sklearn*.

In [7]:
def df_split(df):
    features = df.drop(['id', 'product'], axis=1)
    target = df['product']
    features_train, features_valid, target_train, target_valid = train_test_split(
    features, target, test_size=.25, random_state = rs)
    # размер валидационной выборки уствановим в 25%
    return features_train, features_valid, target_train, target_valid

In [8]:
features_train_0, features_valid_0, target_train_0, target_valid_0 = df_split(geo_data_0)
features_train_1, features_valid_1, target_train_1, target_valid_1 = df_split(geo_data_1)
features_train_2, features_valid_2, target_train_2, target_valid_2 = df_split(geo_data_2)

In [9]:
split_list = [features_train_0, features_valid_0, target_train_0, target_valid_0,
              features_train_1, features_valid_1, target_train_1, target_valid_1,
              features_train_2, features_valid_2, target_train_2, target_valid_2]

In [10]:
# выведем на экран размерности полученных выборок
for selection in split_list:
    print(selection.shape)

(75000, 3)
(25000, 3)
(75000,)
(25000,)
(75000, 3)
(25000, 3)
(75000,)
(25000,)
(75000, 3)
(25000, 3)
(75000,)
(25000,)


Выборки признаков имеют по 3 столбца, целевой признак - один столбец. Обучающие выборки содержат 75000 строк, валидационные - 25000. Разделение произведено корректно.

## Обучение и проверка модели

Для каждого региона **х** обучим модель класса '*линейная регрессия*' с соответствующим индексом **х**. Предсказания модели сохраним в переменной *predictions*_**х**.
В переменной *result*_**x** сохраним показатель среднеквадратической ошибки *RMSE* для целевого признака валидационной выборки и предсказаний модели.

### Регион geo_data_0

In [11]:
model_0 = LinearRegression()
model_0.fit(features_train_0, target_train_0)
predictions_0 = pd.Series(model_0.predict(features_valid_0))
result_0 = mse(target_valid_0, predictions_0)**.5

In [12]:
print(result_0)
print(predictions_0.mean())

37.78865016608396
92.6289007415327


### Регион geo_data_1

In [13]:
model_1 = LinearRegression()
model_1.fit(features_train_1, target_train_1)
predictions_1 = pd.Series(model_1.predict(features_valid_1))
result_1 = mse(target_valid_1, predictions_1)**.5

In [14]:
print(result_1)
print(predictions_1.mean())

0.893713849254031
68.9514776931662


### Регион geo_data_2

In [15]:
model_2 = LinearRegression()
model_2.fit(features_train_2, target_train_2)
predictions_2 = pd.Series(model_2.predict(features_valid_2))
result_2 = mse(predictions_2, target_valid_2)**.5

In [16]:
print(result_2)
print(predictions_2.mean())

39.89052587928214
94.8774353818318


Соберем полученные результаты в таблицу `comparison_of_results`.

In [17]:
predictions_list = [predictions_0, predictions_1, predictions_2]

In [18]:
#for prediction in predictions_list:
#    print(prediction.shape)

In [19]:
data = {'value':['rmse', 'predictions_mean', 'product_min', 'product_max', 'product_mean'],
        'geo_data_0':[result_0, predictions_0.mean(), geo_data_0['product'].min(),
                      geo_data_0['product'].max(), geo_data_0['product'].mean()],
        'geo_data_1': [result_1, predictions_1.mean(), geo_data_1['product'].min(),
                       geo_data_1['product'].max(), geo_data_1['product'].mean()],
        'geo_data_2': [result_2, predictions_2.mean(), geo_data_2['product'].min(),
                       geo_data_2['product'].max(), geo_data_2['product'].mean()]}
        

    
comparison_of_results = pd.DataFrame(data)

In [20]:
comparison_of_results

,value,geo_data_0,geo_data_1,geo_data_2
0,rmse,37.788650,0.893714,39.890526
1,predictions_mean,92.628901,68.951478,94.877435
2,product_min,0.000000,0.000000,0.000000
3,product_max,185.364347,137.945408,190.029838
4,product_mean,92.500000,68.825000,95.000000


Таким образом, мы получили: 
- минимальное значение целевого признака для всех регионов равно 0;
- максимальное значение целевого признака у региона с индексом 2, на втором месте регион 0 (с небольшим отставанием);
- среднее значение предсказанний также наибольшее у региона 2, чуть меньше у реиона 0;
- показатель средней квадратической ошибки минимален у региона 1 (на фоне двух других регионов аномально мал), на втором местое регион 0, на 3 - регион 2.

*(Чем меньше RMSE, тем "кучнее" результаты относительно среднего значения.)*

## Подготовка к расчёту прибыли

In [21]:
#общий бюджет на разработку - 10 млрд. руб.
total_budget = 10**10
# ожидаемая прибыль от единицы 'product' (тысячи баррелей) - 450 тыс. руб.
revenue_per_thousand_barrels = 450*10**3
# количество месторождений для разработки - 200
number_of_wells = 200

Рассчитаем пороговый уровень безубыточности.

In [22]:
quantity_threshold = total_budget / (revenue_per_thousand_barrels * number_of_wells)

In [23]:
quantity_threshold

111.11111111111111

Таким образом, средний показатель запасов месторождений в регионе должен составлять чуть больше 111 тысяч баррелей (*'product' == 111.(1)*) для работы "в 0" (т.е. для равенства доходов расходам и получения нулевой прибыли). Заниматься разработкой месторождений, имеющих меньшие запасы, нецелесообразно с экономической точки зрения.

## Расчёт прибыли и рисков 

In [24]:
# с целью достижения соответствия индексов target_valid 
# и predictions, выполним сброс индексов
target_valid_0 = target_valid_0.reset_index(drop=True)
target_valid_1 = target_valid_1.reset_index(drop=True)
target_valid_2 = target_valid_2.reset_index(drop=True)

Напишем функцию `revenue`, которая принимает на вход три параметра, производит сортировку одного из параметров по убыванию, производит подбор значений второго параметра по соответствию индексов с индексами первого параметра в количестве величины третьего параметра.
Возвращает функция значение формулы: **разница произведения суммарных запасов по n точкам с доходностью от единицы объема и общих затрат на разработку**.
Т.е. потенциальный доход мину снеобходимые расходы, получаем операционную прибыль.

In [25]:
def revenue(target, predictions, count):
    predictions_sorted = pd.Series(predictions).sort_values(ascending=False)
    selected = target[predictions_sorted.index][:count]
    return revenue_per_thousand_barrels * selected.sum() - total_budget

Напишем функцию `revenue_bootstrap`, которая методом "бутстреп" проанализирует 1000 выборок из валидационной переменной целевого признака в размере 500 объектов. В теле функции обратимся к написанной ранее функции `revenue`, а также вычислим риск убытка, равный процентному соотношению количества отрицательных значений к общему количеству значений показателя "выручка".
Функция возвращает средний показатель прибыли, нижнюю и верхнюю границы 95%-го довери тельного интервала, а также риск убытка.

In [26]:
state = np.random.RandomState(rs)
def revenue_bootstrap(target, predictions, count):    
    values = []
    loss_counter = 0
    for i in range(1000):
        target_subsample = target.sample(n=count, replace=True, random_state=state)
        predictions_subsample = predictions[target_subsample.index] 
        revenue_result = revenue(target_subsample, predictions_subsample, number_of_wells)
        values.append(revenue_result)
        if revenue_result  < 0:
            loss_counter += 1
    values = pd.Series(values)
    lower = values.quantile(.025)
    upper = values.quantile(.975)
    mean = values.mean()
    return mean, lower, upper, (loss_counter/len(values)*100)

Вызовем функцию для данных по каждому региону.

In [27]:
mean_0, lower_0, upper_0, loss_risk_0 = revenue_bootstrap(target_valid_0, predictions_0, 500)
print('Средняя выручка:', mean_0)
print('2,5%-квантиль:', lower_0)
print('97,5%-квантиль:', upper_0)
print('Риск получения убытка', loss_risk_0, '%')

Средняя выручка: 468058908.2730077
2,5%-квантиль: -57987078.646888725
97,5%-квантиль: 1001134135.1205382
Риск получения убытка 4.8 %


In [28]:
mean_1, lower_1, upper_1, loss_risk_1 = revenue_bootstrap(target_valid_1, predictions_1, 500)
print("Средняя выручка:", mean_1)
print("2,5%-квантиль:", lower_1)
print("97,5%-квантиль:", upper_1)
print('Риск получения убытка', loss_risk_1, '%')

Средняя выручка: 528396076.1060662
2,5%-квантиль: 123761468.13346158
97,5%-квантиль: 925818986.6239607
Риск получения убытка 0.3 %


In [29]:
mean_2, lower_2, upper_2, loss_risk_2 = revenue_bootstrap(target_valid_2, predictions_2, 500)
print("Средняя выручка:", mean_2)
print("2,5%-квантиль:", lower_2)
print("97,5%-квантиль:", upper_2)
print('Риск получения убытка', loss_risk_2, '%')

Средняя выручка: 348170433.9889426
2,5%-квантиль: -172554696.08704236
97,5%-квантиль: 886473067.2536688
Риск получения убытка 9.1 %


Соберем полученные данные в таблицу `research_result`.

In [30]:
data_result = {'value':['revenue_mean', 'revenue_total', '0.025_quantile',
                        '0.975_quantile', 'loss_risk'],
        'geo_data_0':[mean_0, mean_0*number_of_wells, lower_0, upper_0, loss_risk_0],
        'geo_data_1': [mean_1, mean_1*number_of_wells, lower_1, upper_1, loss_risk_1],
        'geo_data_2': [mean_2, mean_2*number_of_wells, lower_2, upper_2, loss_risk_2]}
        

    
research_result = pd.DataFrame(data_result)

In [31]:
pd.set_option('display.float_format', '{:.2f}'.format)

In [32]:
research_result

,value,geo_data_0,geo_data_1,geo_data_2
0,revenue_mean,468058908.27,528396076.11,348170433.99
1,revenue_total,93611781654.60,105679215221.21,69634086797.79
2,0.025_quantile,-57987078.65,123761468.13,-172554696.09
3,0.975_quantile,1001134135.12,925818986.62,886473067.25
4,loss_risk,4.80,0.30,9.10


Мы получили риски получения убытка в размере 4.8% и 9.1% для регионов 0 и 2 соответственно, т.о. данные регионы не удовлетворяют условию о величине риска не более 2.5%.

Риск получения убытка для региона 1 составляет 0.3%, среднее значение прибыли составляет 528 млн. руб. (что также является наибольшим показателем среди регионов).


Границы доверительного интервала: от 123 млн. до 925 млн. (в таком диапазоне находится потенциальная прибыль от разработки 95% месторождений региона).

На основании проведенного анализа, рекомендуется к разработке регион **geo_data_1**.